In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=gemini_api_key
)

In [4]:
from langchain_core.messages import HumanMessage
llm.invoke([HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?")])

AIMessage(content="저는 실시간 주가 정보를 직접 제공할 수 없습니다. 제 정보는 특정 시점까지 학습된 것이며, 주식 시장은 실시간으로 변동하기 때문입니다.\n\n테슬라(TSLA)의 지난 한 달간 주가 변동을 확인하시려면, 다음 방법들을 이용하시는 것이 가장 정확합니다:\n\n1.  **금융 정보 웹사이트:** 구글 파이낸스, 네이버 금융, 카카오 증권, 인베스팅닷컴(Investing.com), 야후 파이낸스(Yahoo Finance) 등에서 'TSLA'를 검색하시면 지난 한 달간의 주가 차트를 확인하실 수 있습니다.\n2.  **증권사 앱 또는 웹사이트:** 사용하시는 증권사의 앱이나 웹사이트에서도 실시간 및 과거 주가 정보를 조회하실 수 있습니다.\n\n이러한 플랫폼들은 그래프 형태로 주가 추이를 시각적으로 보여주므로, 한 달 전 대비 상승했는지 하락했는지 쉽게 파악하실 수 있습니다.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--9fb84e7c-cc59-408c-8564-890cb46edba8-0', usage_metadata={'input_tokens': 17, 'output_tokens': 948, 'total_tokens': 965, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 738}})

In [5]:
from langchain_core.tools import tool
from datetime import datetime
from zoneinfo import ZoneInfo

@tool
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수 """

    target_tz = ZoneInfo(timezone)
    now = datetime.now(tz=target_tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time=f"{timezone} ({location}) 현재시각 {now}"

    return location_and_local_time


In [7]:
# 도구에서 사용할 스키마 정의
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: TSLA)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [8]:
import yfinance as yf

tsla = yf.Ticker("TSLA")

hist = tsla.history(period="1y")

In [9]:
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-09-20 00:00:00-04:00,241.520004,243.990005,235.919998,238.250000,99879100,0.0,0.0
2024-09-23 00:00:00-04:00,242.610001,250.000000,241.919998,250.000000,86927200,0.0,0.0
2024-09-24 00:00:00-04:00,254.080002,257.190002,249.050003,254.270004,88491000,0.0,0.0
2024-09-25 00:00:00-04:00,252.539993,257.049988,252.279999,257.019989,65034300,0.0,0.0
2024-09-26 00:00:00-04:00,260.600006,261.750000,251.529999,254.220001,67142200,0.0,0.0
...,...,...,...,...,...,...,...
2025-09-15 00:00:00-04:00,423.130005,425.700012,402.429993,410.040009,163823700,0.0,0.0
2025-09-16 00:00:00-04:00,414.500000,423.250000,411.429993,421.619995,104285700,0.0,0.0
2025-09-17 00:00:00-04:00,415.750000,428.309998,409.670013,425.859985,106133500,0.0,0.0


In [12]:
@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """ 주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown() 

    return history_md

In [13]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# 도구들을 tools 리스트에 추가
tools = [get_current_time, get_yf_stock_history]

# 에이전트 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    ("human", "{input}"),
    # AgentExecutor는 agent_scratchpad를 자동으로 채워 넣어,
    # 에이전트가 이전 단계의 행동과 결과를 기억하고 다음 행동을 결정할 수 있도록 함
    ("placeholder", "{agent_scratchpad}"),
])

# 에이전트 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 초기화
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
# 메시지 형식 변경 (agent_executor에 맞게)
messages = [
    HumanMessage("테슬라는 한달 전에 비해 주가가 올랐나 내렸나?"),
]

# agent_executor를 사용하여 실행
response = agent_executor.invoke({"input": messages})



> Entering new AgentExecutor chain...

Invoking: `get_yf_stock_history` with `{'stock_history_input': {'period': '1mo', 'ticker': 'TSLA'}}`


| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|
| 2025-08-20 00:00:00-04:00 | 329.22 | 331.37 | 314.6  |  323.9  | 7.74818e+07 |           0 |              0 |
| 2025-08-21 00:00:00-04:00 | 322.08 | 324.9  | 318.68 |  320.11 | 5.57444e+07 |           0 |              0 |
| 2025-08-22 00:00:00-04:00 | 321.66 | 340.25 | 319.69 |  340.01 | 9.40163e+07 |           0 |              0 |
| 2025-08-25 00:00:00-04:00 | 338.9  | 349.53 | 335.03 |  346.6  | 8.667e+07   |           0 |              0 |
| 2025-08-26 00:00:00-04:00 | 344.93 | 351.9  | 343.72 |  351.67 | 7.66516e+07 |           0 |              0 |
| 2025-08-27 00:00:00-04:00 | 351.94 | 355.39 | 349.16 |  349.6  | 6.551

In [16]:
response

{'input': [HumanMessage(content='테슬라는 한달 전에 비해 주가가 올랐나 내렸나?', additional_kwargs={}, response_metadata={})],
 'output': '테슬라 주가는 한 달 전인 2025년 8월 20일 323.9달러에서 2025년 9월 19일 426.07달러로 올랐습니다.'}